In [40]:
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np

from sklearn.model_selection import train_test_split

def get_weight(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))
def get_Bias(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))
def get_conv2d(X,W,B):
    return tf.nn.conv2d(tf.matmul(X,W), strides = [1,1,1,1])+B
def create_Layer(shape):
    #print(shape[-2])
    return get_weight(shape), get_Bias([shape[-1]])

def Conv2d(X, W, B):
    return tf.nn.conv2d(X,W, strides=[1,1,1,1], padding = 'SAME')+B
def maxPoolAndRelu(T):
    return tf.nn.max_pool(tf.nn.relu(T),strides=[1,2,2,1], ksize=[1,2,2,1], padding='SAME')
def maxPoolAndRelu2(T):
    return tf.nn.max_pool(tf.nn.relu(T),strides=[1,1,1,1], ksize=[1,2,2,1], padding='SAME')
def fullLayer(input, size):
    in_size = int(input.get_shape()[1])
    W = get_weight([in_size,size])
    B = get_Bias([size])
    return tf.matmul(input,W)+B
#data inputif 
#def getNextBatch(data, )

def toNumpy(this,div,shape,type):
    this=this.to_numpy()
    this=np.divide(this,div)
    this=this.reshape(shape)
    this=this.astype(type)
    return this

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [41]:

if __name__ == '__main__':
    data = pd.read_csv('.\\data\\train.csv')
    
    
    yData = data['label']
    del data['label']
    print(data.shape)
    
    x_train, x_test, y_train, y_test = train_test_split(data, yData, test_size=0.03, random_state=42)
    x_train2, x_test2, y_train2, y_test2 = train_test_split(data, yData, test_size=0.2, random_state=42)

    x_train=toNumpy(x_train,255,[-1,28,28,1],'float32')
    x_test2=toNumpy(x_test2,255,[-1,28,28,1],'float32')
    
    y_train = y_train.to_numpy()
    y_train = np.eye(10)[y_train]
    
    y_test2 = y_test2.to_numpy()
    y_test2 = np.eye(10)[y_test2]
    #print(x_train)    
    
    X = tf.placeholder(shape=[None,28,28,1], dtype=tf.float32)
    Y = tf.placeholder(shape=[None,10], dtype=tf.float32)

(42000, 784)


In [42]:
    #Layer!
    W1, B1 = create_Layer([5,5,1,32])
    conv1 = Conv2d(X,W1,B1)
    #conv1 = Conv2d(x_train,W1,B1)
    conv1 = maxPoolAndRelu(conv1)    

In [43]:
    #Layer2.5
    W2, B2 = create_Layer([5,5,32,48])
    conv2 = Conv2d(conv1,W2,B2)
    conv2 = maxPoolAndRelu2(conv2)
    print(conv2)
    
    #Layer2
    W3, B3 = create_Layer([5,5,48,64])
    conv2 = Conv2d(conv2,W3,B3)
    conv2 = maxPoolAndRelu(conv2)
    print(conv2)

Tensor("MaxPool_15:0", shape=(?, 14, 14, 48), dtype=float32)
Tensor("MaxPool_16:0", shape=(?, 7, 7, 64), dtype=float32)


In [44]:
    conv2 = tf.reshape(conv2,shape=[-1,7*7*64])
    print(conv2)
    #1 fully connected
    #conv2 = tf.reshape(conv2,shape=[7*7*64,1024])
    W_fc1 = get_weight([7 * 7 * 64, 1024])
    b_fc1 = get_Bias([1024])
    conv2 = tf.nn.relu(tf.matmul(conv2,W_fc1)+b_fc1)
    print(conv2)
    
    #2 fully connected
    W_fc2 = get_weight([1024, 10])
    b_fc2 = get_Bias([10])
    outPut = tf.matmul(conv2,W_fc2)+b_fc2
    
    
    keep_prob = tf.placeholder(tf.float32)
    #outPut = tf.nn.dropout(outPut, keep_prob)

    print(outPut)

Tensor("Reshape_5:0", shape=(?, 3136), dtype=float32)
Tensor("Relu_22:0", shape=(?, 1024), dtype=float32)
Tensor("add_29:0", shape=(?, 10), dtype=float32)


In [45]:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = outPut, labels = Y))
    
    optimizer=tf.train.GradientDescentOptimizer(0.001).minimize(loss)
    
    y_ = tf.placeholder(shape=[None,10], dtype=tf.float32)

In [ ]:
    start = 0
    size = 256

    #print(y_test)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(200):
            start=0
            for j in range(200):#while( start+size < x_train.shape[0]):
                x_value, y_value = next_batch(size,x_train,y_train)
                #x_value = x_train[start:start+size]
                #y_value = y_train[start:start+size]

                optimizer.run(feed_dict={X:x_value, Y:y_value, keep_prob: 0.5})
                #start+=size
                
        #if i%100==0:
            test = outPut.eval(feed_dict={X:x_test2, keep_prob: 1.0})
            a = tf.argmax(y_test2,1).eval()
            b = tf.argmax(test,1).eval()
            #print('label   ', a)
            #print('expected', b)
            kk = tf.reduce_mean(tf.cast(tf.equal(a,b),dtype=tf.float32))
            print(kk.eval())
            print('')
                
            if(kk.eval() > 0.93):
                test = pd.read_csv('.\\data\\test.csv')
                test=toNumpy(test,255,[-1,28,28,1],'float32')
                tt = tf.argmax(outPut.eval(feed_dict={X:test, keep_prob: 1.0}),1)
                np.savetxt('my.txt',tt.eval(),fmt='%d',delimiter='\n')


0.8457143

